In [5]:
from pathlib import Path
ROOT = Path('~/snow/datasets/').expanduser().absolute()
[d.name for d in ROOT.iterdir() if d.is_dir()]

['d3football.com',
 'www.vgchartz.com',
 'www.cia.gov',
 'www.nndb.com',
 'flightaware.com',
 'itunes.apple.com',
 'seatgeek.com',
 'www.amoeba.com',
 'data.bls.gov']

In [15]:
%%time
dataset_name, run_name = 'd3football.com', 'X'

def read_eval_scores(fname):
    with open(fname) as fo:
        lines = fo.readlines()
        try:
            ind = lines.index('Overall Performance:\n')
            scores = [float(l.split(' ')[1]) for l in lines[ind+1:ind+4]]
        except:
            scores = (0,0,0)
        return dict(zip('prf', scores))
    
# import subprocess
# cmd = 'bash', 'evaluate_containment_specific', f'datasets/{dataset_name}', run_name
# eval_out = subprocess.run(cmd, cwd=ROOT.parent, check=True, capture_output=True)
# print(eval_out.stderr.decode())

print(read_eval_scores(ROOT.joinpath(f'{dataset_name}/evaluate_containment_{run_name}.log')))

{'p': 0.961633, 'r': 0.136782, 'f': 0.239499}
CPU times: user 1.64 ms, sys: 1.73 ms, total: 3.38 ms
Wall time: 2.52 ms


In [113]:
# Load global log results
import pandas as pd

snowcodes = {'R':'Schema', 'B':'Binary', 'S_noContextColumns':'N-Ary', 'S':'N-ary+C', 'M':'Match', 'M_norm':'Norm'}

header = 'domain run pr re f1 extractedEntities referenceEntities extractedTuples referenceTuples'.split()
resultsdf = pd.read_csv(ROOT.joinpath('evaluation_results_containment.tsv'), sep='\t', names=header)
resultsdf.run = resultsdf.run.str[11:-13].replace(snowcodes)
resultsdf = resultsdf.set_index(['domain', 'run'])
resultsdf = resultsdf[~resultsdf.index.duplicated(keep='last')].sort_index()
resultsdf = resultsdf[['pr', 're', 'f1']].unstack(level=1)
resultsdf.loc['Macro Avg.'] = resultsdf.mean()

display(resultsdf.style.format("{:.3}"))
# print(resultsdf.to_latex(float_format='%.3f'))

## Inspect outputs

In [109]:
dataset_name, run_name = 'data.bls.gov', 'X-9mar-3'

In [110]:
import takco
fd_path = ROOT.joinpath(f'{dataset_name}/normalised_{run_name}_fd_relations')
fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(fd_path.glob("*.json")))
takco.TableSet.dataset(fd_dataset).preview(ntables=None)

?,0,1
,phone,FK
,2428870.0,AdministrativeRegion_data.bls.gov~Row33
,4654510.0,AdministrativeRegion_data.bls.gov~Row8
,7711173.0,AdministrativeRegion_data.bls.gov~Row26
,6823190.0,AdministrativeRegion_data.bls.gov~Row29
,2621856.0,AdministrativeRegion_data.bls.gov~Row7
?,0,1
,postal abbr,FK
,ne,AdministrativeRegion_data.bls.gov~Row27
,nv,AdministrativeRegion_data.bls.gov~Row15


In [111]:
import takco
fd_path = ROOT.joinpath(f'{dataset_name}/evaluation/normalised_fd_relations')
fd_dataset = takco.evaluate.dataset.WebDataCommons(fnames=list(fd_path.glob("*.json")))
takco.TableSet.dataset(fd_dataset).preview(ntables=None)

?,0,1,2,3,4
,page title (AdministrativeRegion),uri 2 (year),uri 3 (month),location quotient (9),FK
,"separating, filtering, clarifying, precipitating, (...)",2011-01-01T00:00,may,1.35,AdministrativeRegion_data.bls.gov~Row16
,"separating, filtering, clarifying, precipitating, (...)",2011-01-01T00:00,may,1.78,AdministrativeRegion_data.bls.gov~Row20
,"separating, filtering, clarifying, precipitating, (...)",2011-01-01T00:00,may,1.52,AdministrativeRegion_data.bls.gov~Row0
,"separating, filtering, clarifying, precipitating, (...)",2011-01-01T00:00,may,0.87,AdministrativeRegion_data.bls.gov~Row8
,"health and safety engineers, except mining safety (...)",2012-01-01T00:00,may,0.98,AdministrativeRegion_data.bls.gov~Row10
?,0,1,2,3,4
,page title (AdministrativeRegion),uri 2 (year),uri 3 (month),annual mean wage,FK
,veterinary technologists and technicians,2008-01-01T00:00,may,29270.0,AdministrativeRegion_data.bls.gov~Row31
,veterinary technologists and technicians,2008-01-01T00:00,may,29390.0,AdministrativeRegion_data.bls.gov~Row16
